In [ ]:
## Cell 1: Install deps & Authenticate Kaggle
# Install PyTorch, OpenCV, Kaggle API, tqdm, scikit‑learn
!pip install --quiet torch torchvision torchaudio opencv-python kaggle tqdm scikit-learn

# Upload your kaggle.json (drag & drop in the Files pane)
from google.colab import files
files.upload()  # select kaggle.json

# Move token into place
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json


In [ ]:
## Cell 2: Download & Unzip CIFake
# Download & unzip into /content/dataset
!kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images \
    -p /content/dataset --unzip


Dataset URL: https://www.kaggle.com/datasets/birdy654/cifake-real-and-ai-generated-synthetic-images
License(s): other
  0% 0.00/105M [00:00<?, ?B/s]
100% 105M/105M [00:00<00:00, 1.30GB/s]


In [ ]:
# Cell 3: No split needed—just load from existing train/ & test/ folders

TRAIN_DIR = "/content/dataset/train"
TEST_DIR  = "/content/dataset/test"

import os
print("Train classes:", sorted(os.listdir(TRAIN_DIR)))
print("Test  classes:", sorted(os.listdir(TEST_DIR)))
print("Train samples:",
      sum(len(files) for _,_,files in os.walk(TRAIN_DIR)))
print("Test  samples:",
      sum(len(files) for _,_,files in os.walk(TEST_DIR)))


Train classes: ['FAKE', 'REAL']
Test  classes: ['FAKE', 'REAL']
Train samples: 100000
Test  samples: 20000


In [ ]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Settings
BATCH_SIZE = 32
EPOCHS = 1
LR = 1e-4
USE_AMP = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transforms
transform = transforms.Compose([
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Data Loaders
train_ds = ImageFolder(TRAIN_DIR, transform=transform)
test_ds  = ImageFolder(TEST_DIR,  transform=transform)

train_ld = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
test_ld  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("Class-to-Index:", train_ds.class_to_idx)
print(f"Train: {len(train_ds)} samples | Test: {len(test_ds)} samples\n")

# Load pretrained ResNet-50
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scaler = torch.cuda.amp.GradScaler() if USE_AMP else None

# Training loop
start_time = time.time()
for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0
    loop = tqdm(train_ld, desc=f"Epoch {epoch}/{EPOCHS}", ncols=100)

    for imgs, labels in loop:
        imgs = imgs.to(DEVICE)
        labels = labels.float().unsqueeze(1).to(DEVICE)

        optimizer.zero_grad()
        if USE_AMP:
            with torch.cuda.amp.autocast():
                outputs = model(imgs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=f"{loss.item():.4f}")

    # Validation
    model.eval()
    preds, gts = [], []
    with torch.no_grad():
        for imgs, labels in test_ld:
            imgs = imgs.to(DEVICE)
            outputs = model(imgs)
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            preds.extend((probs > 0.5).astype(int).tolist())
            gts.extend(labels.numpy().tolist())

    acc = accuracy_score(gts, preds)
    print(f"✅ Epoch {epoch} finished — Loss: {running_loss/len(train_ld):.4f}, "
          f"Test Accuracy: {acc*100:.2f}%")

total_time = (time.time() - start_time) / 60
print(f"\n✔ Training complete in {total_time:.2f} minutes.")

# Save model
#torch.save(model.state_dict(), "/content/cnn_detection_final.pth")
print("✔ Model saved to /content/cnn_detection_final.pth")


Class-to-Index: {'FAKE': 0, 'REAL': 1}
Train: 100000 samples | Test: 20000 samples



/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipython-input-21-3983386504.py:41: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if USE_AMP else None
Epoch 1/1:   0%|                                                           | 0/3125 [00:00<?, ?it/s]/tmp/ipython-input-21-3983386504.py:56: FutureWarn

✅ Epoch 1 finished — Loss: 0.1661, Test Accuracy: 95.76%

✔ Training complete in 3.07 minutes.
✔ Model saved to /content/cnn_detection_final.pth


In [ ]:
torch.save(model.state_dict(), "/content/cnn_detection_final.pth")

In [ ]:
from PIL import Image

def predict_image(image_path):
    # Load model
    model = models.resnet50(weights=None)  # no pretrained weights
    model.fc = nn.Linear(model.fc.in_features, 1)
    model.load_state_dict(torch.load("/content/cnn_detection_final.pth", map_location=DEVICE))
    model = model.to(DEVICE)  # 🔧 move to correct device
    model.eval()

    # Image preprocessing
    transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor()
    ])
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(DEVICE)  # 🔧 move image to same device

    # Inference
    with torch.no_grad():
        output = model(img_tensor)
        prob = torch.sigmoid(output).item()

    print(f"🧠 Prediction: {'FAKE' if prob > 0.5 else 'REAL'} (confidence: {prob:.4f})")

# Example usage:
predict_image("/content/t2.png")

🧠 Prediction: REAL (confidence: 0.0000)
